# 🎨 Projet : Compression d'Images avec SVD

## 🎯 Objectif du Projet

Dans ce projet, tu vas :
- 📸 Charger une image (ou en générer une)
- 🔬 Appliquer la **décomposition SVD**
- 🗜️ Reconstruire l'image avec différents **rangs**
- 📊 Analyser le **compromis qualité vs compression**
- 💾 Calculer les **taux de compression**
- 🎨 Visualiser les résultats

## 💡 Pourquoi c'est Important ?

La compression d'images par SVD est utilisée dans :
- **JPEG** : Utilise une technique similaire (DCT)
- **Netflix** : Recommandations via matrix factorization
- **Machine Learning** : Feature extraction et dimensionality reduction
- **Traitement d'images** : Denoising et compression

## 🔥 Ce que tu vas Apprendre

1. Comment la SVD décompose une image en composantes principales
2. Le compromis entre qualité et taille de stockage
3. L'importance des valeurs singulières
4. Applications pratiques de l'algèbre linéaire

---

In [ ]:
# 📦 Imports
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO

plt.style.use('seaborn-v0_8-darkgrid')
np.set_printoptions(precision=4, suppress=True)

print("✅ Imports réussis !")

---
## 📸 Étape 1 : Charger ou Créer une Image

On va créer une image synthétique simple pour bien comprendre le processus.

In [ ]:
# Option 1 : Créer une image synthétique

def create_gradient_image(size=(200, 200)):
    """Crée une image avec un gradient."""
    x = np.linspace(0, 1, size[1])
    y = np.linspace(0, 1, size[0])
    X, Y = np.meshgrid(x, y)
    
    # Patron circulaire
    image = np.sin(10 * X) * np.cos(10 * Y)
    image = (image - image.min()) / (image.max() - image.min())  # Normaliser à [0,1]
    
    return image

def create_geometric_image(size=(200, 200)):
    """Crée une image avec des formes géométriques."""
    image = np.zeros(size)
    
    # Rectangle
    image[50:150, 50:100] = 1.0
    
    # Cercle
    center = (100, 150)
    radius = 30
    y, x = np.ogrid[:size[0], :size[1]]
    mask = (x - center[1])**2 + (y - center[0])**2 <= radius**2
    image[mask] = 0.7
    
    return image

# Choisis ton image
print("Choix d'image :")
print("1. Gradient avec motif sinusoïdal")
print("2. Formes géométriques simples")

# Créer l'image (change ici pour essayer différentes images)
image = create_gradient_image(size=(200, 200))
# image = create_geometric_image(size=(200, 200))

# Visualiser
plt.figure(figsize=(8, 6))
plt.imshow(image, cmap='gray')
plt.colorbar()
plt.title('Image Originale', fontsize=14, fontweight='bold')
plt.axis('off')
plt.show()

print(f"\n📊 Dimensions de l'image : {image.shape}")
print(f"📏 Taille en mémoire : {image.nbytes} bytes ({image.nbytes/1024:.2f} KB)")

### 📸 Option 2 : Charger une Vraie Image

Si tu veux utiliser une vraie image, décommente le code ci-dessous.

In [ ]:
# # Charger une image depuis un fichier local
# from PIL import Image

# # Remplace par le chemin de ton image
# image_path = "ton_image.jpg"  # ou .png, .jpeg, etc.

# # Charger et convertir en niveaux de gris
# img = Image.open(image_path).convert('L')

# # Redimensionner si trop grande (optionnel)
# max_size = 400
# if max(img.size) > max_size:
#     img.thumbnail((max_size, max_size), Image.Resampling.LANCZOS)

# # Convertir en array NumPy et normaliser
# image = np.array(img).astype(float) / 255.0

# plt.figure(figsize=(8, 6))
# plt.imshow(image, cmap='gray')
# plt.title('Image Chargée')
# plt.axis('off')
# plt.show()

---
## 🔬 Étape 2 : Appliquer la SVD

On va décomposer notre image en ses composantes principales.

In [ ]:
# SVD de l'image
print("🔬 Application de la SVD...\n")

U, s, Vt = np.linalg.svd(image, full_matrices=False)

print(f"✅ SVD terminée !")
print(f"\n📊 Résultats :")
print(f"   U  : {U.shape} (vecteurs singuliers gauches)")
print(f"   s  : {s.shape} (valeurs singulières)")
print(f"   Vᵀ : {Vt.shape} (vecteurs singuliers droits transposés)")

# Nombre de composantes
n_components = len(s)
print(f"\n🔢 Nombre total de composantes : {n_components}")

# Visualiser les valeurs singulières
plt.figure(figsize=(12, 4))

# Graphique 1 : Valeurs singulières
plt.subplot(1, 2, 1)
plt.plot(s, 'o-', linewidth=2, markersize=4)
plt.xlabel('Index', fontsize=11)
plt.ylabel('Valeur Singulière', fontsize=11)
plt.title('Spectre des Valeurs Singulières', fontsize=12, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.yscale('log')

# Graphique 2 : Énergie cumulée
plt.subplot(1, 2, 2)
energy = np.cumsum(s**2) / np.sum(s**2)
plt.plot(energy * 100, linewidth=2)
plt.axhline(y=90, color='r', linestyle='--', alpha=0.7, label='90% d\'énergie')
plt.axhline(y=95, color='g', linestyle='--', alpha=0.7, label='95% d\'énergie')
plt.axhline(y=99, color='b', linestyle='--', alpha=0.7, label='99% d\'énergie')
plt.xlabel('Nombre de Composantes', fontsize=11)
plt.ylabel('Énergie Conservée (%)', fontsize=11)
plt.title('Énergie Cumulée', fontsize=12, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()

# Statistiques
print(f"\n📈 Statistiques :")
print(f"   Plus grande valeur singulière : {s[0]:.4f}")
print(f"   Plus petite valeur singulière : {s[-1]:.4f}")
print(f"   Ratio : {s[0]/s[-1]:.2f}")

# Combien de composantes pour 90%, 95%, 99% d'énergie ?
for threshold in [0.90, 0.95, 0.99]:
    k = np.argmax(energy >= threshold) + 1
    print(f"   {k:3d} composantes pour {threshold*100:.0f}% d'énergie ({k/n_components*100:.1f}% du total)")

---
## 🗜️ Étape 3 : Reconstruction avec Différents Rangs

Voyons comment l'image change quand on garde plus ou moins de composantes.

In [ ]:
def reconstruct_image(U, s, Vt, k):
    """Reconstruit l'image en gardant seulement k composantes."""
    Sigma_k = np.zeros((k, k))
    Sigma_k[:k, :k] = np.diag(s[:k])
    return U[:, :k] @ Sigma_k @ Vt[:k, :]

def calculate_compression_ratio(original_shape, k):
    """Calcule le taux de compression."""
    m, n = original_shape
    original_size = m * n
    compressed_size = k * (m + n + 1)  # U[:, :k] + s[:k] + Vt[:k, :]
    return original_size / compressed_size

def calculate_error(original, reconstructed):
    """Calcule l'erreur de reconstruction."""
    return np.linalg.norm(original - reconstructed, 'fro') / np.linalg.norm(original, 'fro')

# Rangs à tester
ranks = [1, 5, 10, 20, 50, n_components]

# Filtrer les rangs valides
ranks = [k for k in ranks if k <= n_components]

print(f"🎨 Reconstruction avec différents rangs : {ranks}\n")

# Visualisation
n_ranks = len(ranks)
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

for idx, k in enumerate(ranks):
    # Reconstruction
    reconstructed = reconstruct_image(U, s, Vt, k)
    
    # Métriques
    compression_ratio = calculate_compression_ratio(image.shape, k)
    error = calculate_error(image, reconstructed)
    energy_preserved = np.sum(s[:k]**2) / np.sum(s**2)
    
    # Affichage
    ax = axes[idx]
    ax.imshow(reconstructed, cmap='gray', vmin=0, vmax=1)
    ax.set_title(
        f'Rang {k}\n'
        f'Compression: {compression_ratio:.2f}x\n'
        f'Erreur: {error*100:.2f}% | '
        f'Énergie: {energy_preserved*100:.1f}%',
        fontsize=10
    )
    ax.axis('off')
    
    # Stats
    print(f"Rang {k:3d} :")
    print(f"   Compression : {compression_ratio:6.2f}x")
    print(f"   Erreur      : {error*100:6.2f}%")
    print(f"   Énergie     : {energy_preserved*100:6.2f}%")
    print()

plt.tight_layout()
plt.show()

---
## 📊 Étape 4 : Analyse du Compromis Qualité vs Compression

Traçons les courbes pour bien comprendre le trade-off.

In [ ]:
# Calculer pour tous les rangs possibles
all_ranks = range(1, min(50, n_components) + 1)
compression_ratios = []
errors = []
energies = []

for k in all_ranks:
    reconstructed = reconstruct_image(U, s, Vt, k)
    compression_ratios.append(calculate_compression_ratio(image.shape, k))
    errors.append(calculate_error(image, reconstructed))
    energies.append(np.sum(s[:k]**2) / np.sum(s**2))

# Visualisation
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Erreur vs Rang
axes[0, 0].plot(all_ranks, np.array(errors) * 100, linewidth=2)
axes[0, 0].set_xlabel('Nombre de Composantes')
axes[0, 0].set_ylabel('Erreur de Reconstruction (%)')
axes[0, 0].set_title('Erreur vs Rang', fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# 2. Compression vs Rang
axes[0, 1].plot(all_ranks, compression_ratios, linewidth=2, color='green')
axes[0, 1].set_xlabel('Nombre de Composantes')
axes[0, 1].set_ylabel('Taux de Compression (x)')
axes[0, 1].set_title('Compression vs Rang', fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# 3. Énergie Conservée vs Rang
axes[1, 0].plot(all_ranks, np.array(energies) * 100, linewidth=2, color='orange')
axes[1, 0].axhline(y=90, color='r', linestyle='--', alpha=0.5)
axes[1, 0].axhline(y=95, color='g', linestyle='--', alpha=0.5)
axes[1, 0].axhline(y=99, color='b', linestyle='--', alpha=0.5)
axes[1, 0].set_xlabel('Nombre de Composantes')
axes[1, 0].set_ylabel('Énergie Conservée (%)')
axes[1, 0].set_title('Énergie vs Rang', fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# 4. Pareto : Erreur vs Compression
axes[1, 1].scatter(compression_ratios, np.array(errors) * 100, 
                   c=all_ranks, cmap='viridis', s=50)
axes[1, 1].set_xlabel('Taux de Compression (x)')
axes[1, 1].set_ylabel('Erreur de Reconstruction (%)')
axes[1, 1].set_title('Trade-off Qualité vs Compression\n(couleur = rang)', 
                     fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
plt.colorbar(axes[1, 1].collections[0], ax=axes[1, 1], label='Rang')

plt.tight_layout()
plt.show()

# Point optimal (exemple : 95% d'énergie)
target_energy = 0.95
optimal_k = np.argmax(np.array(energies) >= target_energy) + 1
optimal_compression = compression_ratios[optimal_k - 1]
optimal_error = errors[optimal_k - 1]

print(f"\n🎯 Point Optimal (95% d'énergie) :")
print(f"   Rang          : {optimal_k}")
print(f"   Compression   : {optimal_compression:.2f}x")
print(f"   Erreur        : {optimal_error*100:.2f}%")
print(f"   Gain mémoire  : {(1 - 1/optimal_compression)*100:.1f}%")

---
## 🎨 Étape 5 : Visualisation des Composantes Principales

Regardons à quoi ressemblent les vecteurs singuliers.

In [ ]:
# Visualiser les premières composantes
n_show = min(6, n_components)

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.ravel()

for i in range(n_show):
    # Composante i
    component = np.outer(U[:, i], Vt[i, :])
    
    ax = axes[i]
    im = ax.imshow(component, cmap='RdBu_r')
    ax.set_title(f'Composante {i+1}\nσ = {s[i]:.3f} '
                f'({s[i]**2/np.sum(s**2)*100:.1f}% d\'énergie)',
                fontsize=10)
    ax.axis('off')
    plt.colorbar(im, ax=ax, fraction=0.046)

plt.suptitle('Composantes Principales (vecteurs singuliers)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("\n💡 Observation :")
print("   - Les premières composantes capturent les structures principales")
print("   - Les composantes suivantes capturent les détails fins")
print("   - Chaque composante = produit extérieur d'un vecteur gauche et droit")

---
## 📈 Étape 6 : Comparaison Interactive

Fonction pour comparer facilement différents rangs.

In [ ]:
def compare_reconstructions(original, U, s, Vt, ranks):
    """Compare l'original avec plusieurs reconstructions."""
    n_images = len(ranks) + 1
    fig, axes = plt.subplots(1, n_images, figsize=(4*n_images, 4))
    
    # Image originale
    axes[0].imshow(original, cmap='gray')
    axes[0].set_title('Original\n100%', fontweight='bold')
    axes[0].axis('off')
    
    # Reconstructions
    for idx, k in enumerate(ranks, 1):
        reconstructed = reconstruct_image(U, s, Vt, k)
        error = calculate_error(original, reconstructed)
        compression = calculate_compression_ratio(original.shape, k)
        energy = np.sum(s[:k]**2) / np.sum(s**2)
        
        axes[idx].imshow(reconstructed, cmap='gray', vmin=0, vmax=1)
        axes[idx].set_title(
            f'Rang {k}\n'
            f'{compression:.1f}x | '
            f'{error*100:.1f}% err | '
            f'{energy*100:.0f}% E',
            fontsize=10
        )
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

# Essayer différentes combinaisons
print("Comparaison 1 : Très faible rang")
compare_reconstructions(image, U, s, Vt, [1, 2, 5])

print("\nComparaison 2 : Rang moyen")
compare_reconstructions(image, U, s, Vt, [10, 20, 30])

print("\nComparaison 3 : Rang élevé")
if n_components >= 100:
    compare_reconstructions(image, U, s, Vt, [50, 75, 100])

---
## 💾 Étape 7 : Calcul des Gains de Stockage

Analysons combien d'espace on économise vraiment.

In [ ]:
def storage_analysis(original_shape, ranks, bytes_per_value=8):
    """Analyse détaillée du stockage."""
    m, n = original_shape
    original_bytes = m * n * bytes_per_value
    
    print(f"📊 Analyse de Stockage\n")
    print(f"Image originale : {m} × {n} = {m*n:,} valeurs")
    print(f"Taille originale : {original_bytes:,} bytes ({original_bytes/1024:.2f} KB)\n")
    print("="*70)
    print(f"{'Rang':>6} {'Stockage (KB)':>15} {'Compression':>12} {'Économie %':>12}")
    print("="*70)
    
    for k in ranks:
        # Calcul du stockage compressé
        # U[:, :k] : m × k valeurs
        # s[:k]    : k valeurs
        # Vt[:k, :]: k × n valeurs
        compressed_values = m*k + k + k*n
        compressed_bytes = compressed_values * bytes_per_value
        
        compression_ratio = original_bytes / compressed_bytes
        savings = (1 - compressed_bytes/original_bytes) * 100
        
        print(f"{k:6d} {compressed_bytes/1024:15.2f} {compression_ratio:12.2f}x {savings:11.1f}%")

# Analyse
storage_analysis(
    image.shape,
    ranks=[1, 5, 10, 20, 30, 50, 75, 100][:min(8, n_components)]
)

print("\n💡 Insights :")
print("   - Avec seulement ~10-20% des composantes, on peut avoir 90%+ d'énergie")
print("   - Le gain de compression diminue avec le rang (courbe logarithmique)")
print("   - Trade-off : qualité visuelle vs espace disque")

---
## 🎯 Étape 8 : Challenge - Trouve le Meilleur Compromis

À toi de jouer !

In [ ]:
# 🎯 TON CHALLENGE :
#
# Trouve le rang optimal selon ces critères :
# 1. Compression d'au moins 5x
# 2. Erreur de reconstruction < 5%
# 3. Énergie conservée > 90%
#
# Écris ton code ici :

def find_optimal_rank(U, s, Vt, image, 
                      min_compression=5, 
                      max_error=0.05, 
                      min_energy=0.90):
    """
    Trouve le rang minimal qui satisfait tous les critères.
    
    Retourne: (rang_optimal, stats_dict)
    """
    # TON CODE ICI
    # Indice : parcours les rangs de 1 à n_components
    # et teste chaque critère
    
    pass

# Teste ta fonction
# optimal_k, stats = find_optimal_rank(U, s, Vt, image)
# print(f"\n🏆 Rang optimal trouvé : {optimal_k}")
# print(f"   Stats : {stats}")

### ✅ Solution du Challenge

In [ ]:
# SOLUTION (ne regarde pas avant d'avoir essayé !)

def find_optimal_rank_solution(U, s, Vt, image, 
                               min_compression=5, 
                               max_error=0.05, 
                               min_energy=0.90):
    """Trouve le rang optimal selon les critères."""
    n = len(s)
    
    for k in range(1, n + 1):
        # Calculer les métriques
        compression = calculate_compression_ratio(image.shape, k)
        reconstructed = reconstruct_image(U, s, Vt, k)
        error = calculate_error(image, reconstructed)
        energy = np.sum(s[:k]**2) / np.sum(s**2)
        
        # Tester les critères
        if (compression >= min_compression and 
            error <= max_error and 
            energy >= min_energy):
            
            stats = {
                'compression': compression,
                'error': error,
                'energy': energy
            }
            return k, stats
    
    return None, None

# Test
optimal_k, stats = find_optimal_rank_solution(U, s, Vt, image)

if optimal_k:
    print(f"\n🏆 Rang optimal trouvé : {optimal_k}")
    print(f"\n📊 Statistiques :")
    print(f"   ✅ Compression  : {stats['compression']:.2f}x (≥ 5x)")
    print(f"   ✅ Erreur       : {stats['error']*100:.2f}% (≤ 5%)")
    print(f"   ✅ Énergie      : {stats['energy']*100:.2f}% (≥ 90%)")
    
    # Visualiser
    reconstructed = reconstruct_image(U, s, Vt, optimal_k)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original', fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(reconstructed, cmap='gray')
    axes[1].set_title(f'Compressée (rang {optimal_k})\n'
                     f'{stats["compression"]:.1f}x compression, '
                     f'{stats["error"]*100:.1f}% erreur',
                     fontweight='bold')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("\n❌ Aucun rang ne satisfait tous les critères simultanément.")
    print("   Essaie d'assouplir les contraintes !")

---
## 🎉 Félicitations !

Tu as terminé le projet de compression d'images avec SVD ! 🎊

### 📚 Ce que tu as appris :

✅ **SVD en pratique** : Décomposer une matrice en composantes principales
✅ **Approximation de rang faible** : Réduire la dimension tout en gardant l'essentiel
✅ **Trade-offs** : Comprendre le compromis qualité vs compression
✅ **Analyse quantitative** : Mesurer erreurs, compression, énergie
✅ **Applications réelles** : Voir l'algèbre linéaire en action

### 🚀 Pour aller plus loin :

1. **Images couleur** : Applique SVD sur chaque canal RGB séparément
2. **Vraies photos** : Teste avec tes propres images
3. **Comparaison JPEG** : Compare avec la compression JPEG standard
4. **Denoising** : Utilise SVD pour enlever le bruit d'images bruitées
5. **Vidéo** : Étends à la compression vidéo (SVD sur des matrices 3D)

### 💡 Applications dans le Monde Réel :

- **Netflix** : Recommandations via matrix factorization (SVD)
- **Google Photos** : Reconnaissance d'images avec réduction de dimension
- **Finance** : Analyse de corrélations dans les portefeuilles
- **NLP** : Latent Semantic Analysis (LSA) pour comprendre le texte
- **Traitement du signal** : Filtrage et compression audio

---

**Tu maîtrises maintenant un outil FONDAMENTAL du Machine Learning et de la Data Science ! 💪🔥**

**Continue à explorer et expérimenter ! C'est comme ça qu'on devient expert ! 🚀**